<a href="https://colab.research.google.com/github/aneeq-shaffy/DL-labsheets/blob/main/Pest_solution_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

from PIL import Image
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torchvision.transforms as transforms
import torchvision.models as models


In [7]:
import kagglehub

path = kagglehub.dataset_download("imbikramsaha/paddy-doctor")
print("Dataset path:", path)


Using Colab cache for faster access to the 'paddy-doctor' dataset.
Dataset path: /kaggle/input/paddy-doctor


In [8]:
dataset_root_path = os.path.join(path, 'paddy-disease-classification')
train_csv_path = os.path.join(dataset_root_path, 'train.csv')
train_images_path = os.path.join(dataset_root_path, 'train_images')


In [9]:
full_df = pd.read_csv(train_csv_path)


In [10]:
train_df, val_df = train_test_split(
    full_df,
    test_size=0.1,
    stratify=full_df['label'],
    random_state=42
)

print(len(train_df), len(val_df))


9366 1041


In [11]:
label_counts = train_df['label'].value_counts()
label_order = sorted(train_df['label'].unique())

counts = torch.tensor(
    [label_counts[label] for label in label_order],
    dtype=torch.float
)

class_weights = 1.0 / counts
class_weights = class_weights / class_weights.sum()


In [12]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.5),
    transforms.RandomRotation(20),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.05),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

rare_class_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(0.7),
    transforms.RandomVerticalFlip(0.7),
    transforms.RandomRotation(30),
    transforms.ColorJitter(0.3,0.3,0.3,0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])


In [13]:
class PaddyDiseaseDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None, rare_class_transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform
        self.rare_transform = rare_class_transform

        self.labels = sorted(self.df['label'].unique())
        self.label_to_idx = {l:i for i,l in enumerate(self.labels)}

        self.rare_classes = {
            "bacterial_panicle_blight",
            "bacterial_leaf_streak",
            "downy_mildew"
        }

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row.label, row.image_id)
        label = self.label_to_idx[row.label]

        image = Image.open(img_path).convert("RGB")

        if row.label in self.rare_classes and self.rare_transform:
            image = self.rare_transform(image)
        else:
            image = self.transform(image)

        return image, label


In [14]:
train_dataset = PaddyDiseaseDataset(
    train_df, train_images_path, train_transform, rare_class_transform
)

val_dataset = PaddyDiseaseDataset(
    val_df, train_images_path, val_transform
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)


In [15]:
model = models.efficientnet_b1(
    weights=models.EfficientNet_B1_Weights.DEFAULT
)

for p in model.parameters():
    p.requires_grad = False

in_features = model.classifier[1].in_features
model.classifier = nn.Linear(in_features, len(label_order))


In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)


In [17]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    for images, labels in loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total


def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return running_loss / total, correct / total


In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(
        model, train_loader, optimizer, criterion, device
    )

    val_loss, val_acc = validate(
        model, val_loader, criterion, device
    )

    scheduler.step()

    print(
        f"Epoch [{epoch+1}/{num_epochs}] "
        f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} "
        f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}"
    )


Epoch [1/10] Train Loss: 1.9109 | Train Acc: 0.3915 Val Loss: 1.7421 | Val Acc: 0.4822
Epoch [2/10] Train Loss: 1.6059 | Train Acc: 0.4644 Val Loss: 1.6266 | Val Acc: 0.5168
Epoch [3/10] Train Loss: 1.4992 | Train Acc: 0.4963 Val Loss: 1.5388 | Val Acc: 0.5591


In [ ]:
# Unfreeze last 2 EfficientNet blocks
for name, param in model.named_parameters():
    if "features.6" in name or "features.7" in name:
        param.requires_grad = True


In [ ]:
optimizer = optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-4,
    weight_decay=1e-4
)


In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(
        model, train_loader, optimizer, criterion, device
    )

    val_loss, val_acc = validate(
        model, val_loader, criterion, device
    )

    scheduler.step()

    print(
        f"Epoch [{epoch+1}/{num_epochs}] "
        f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} "
        f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}"
    )


In [ ]:
torch.save({
    "model_state": model.state_dict(),
    "label_to_idx": train_dataset.label_to_idx
}, "paddy_efficientnet_b1.pth")


In [ ]:
class PaddyTestDataset(Dataset):
    def __init__(self, img_dir, transform):
        self.img_dir = img_dir
        self.transform = transform
        self.images = sorted(os.listdir(img_dir))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.img_dir, img_name)

        image = Image.open(img_path).convert("RGB")
        image = self.transform(image)

        return image, img_name


In [ ]:
test_images_path = os.path.join(dataset_root_path, "test_images")

test_dataset = PaddyTestDataset(test_images_path, val_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
model.eval()
predictions = []

idx_to_label = {v:k for k,v in train_dataset.label_to_idx.items()}

with torch.no_grad():
    for images, names in test_loader:
        images = images.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)

        for name, p in zip(names, preds):
            predictions.append((name, idx_to_label[p.item()]))


In [ ]:
pred_df = pd.DataFrame(predictions, columns=["image_id", "label"])
pred_df.to_csv("submission.csv", index=False)

print("Saved submission.csv")


In [ ]:
checkpoint = torch.load("paddy_efficientnet_b1.pth", map_location='cpu')
print(checkpoint['model_state'].keys())


In [ ]:
import torch
from torchvision import models
import torch.nn as nn

# Load checkpoint
checkpoint = torch.load("paddy_efficientnet_b1.pth", map_location='cpu')
num_classes = len(checkpoint['label_to_idx'])

# Create EfficientNetB1 without pretrained weights
model = models.efficientnet_b1(weights=None)

# Get the input features for the classifier
in_features = model.classifier[1].in_features  # should be 1280

# Replace Sequential with plain Linear (matches checkpoint)
model.classifier = nn.Linear(in_features, num_classes)

# Load weights
model.load_state_dict(checkpoint['model_state'])
model.eval()

print("✅ Checkpoint loaded successfully")


In [ ]:
import torch

# Trace model for TorchScript
dummy_input = torch.randn(1, 3, 224, 224)
traced_model = torch.jit.trace(model, dummy_input)
traced_model.save("paddy_efficientnet_b1.pt")

# Convert TorchScript to ONNX first
torch.onnx.export(
    model, dummy_input, "paddy_efficientnet_b1.onnx",
    input_names=['input'], output_names=['output'],
    opset_version=18
)

# Then ONNX → TFLite via onnx-tf or tf2onnx
# (I can give a ready-to-use script for that next)


In [ ]:
pip install ai-edge-torch


In [ ]:
pip install torch efficientnet-pytorch


In [ ]:
checkpoint = torch.load("paddy_efficientnet_b1.pth")
model.load_state_dict(checkpoint['model_state'])
model.eval()
